# How I write code now.

This is a shallow overview of how I write code nowadays. I've come to work this way partly due to what I learned in my classes and partly because I ignored exactly what my classes taught me. I'm not an expert, and I easily get confused about how python interacts with other programs like the terminal, Anaconda, and the OS (Windows10). Also, this is the first time I've used a Jupyter Notebook for this kind of thing, so expect some breakage.

## How I used to do it

I dug around old files of mine and found this example. This is the whole script, written in python 2.

What's wrong with it? A few things.
* input file is hard coded into the script
* does not check what kind of data the text file is
* I've forgotten why I wrote this, and there's no documentation to give me any clues. No comment, no docstrings, nothing.
* results are unceremoniously dumped to the command line
* With respect to style, and the amount of work being done here, I don't need a function.

## What I was trying to do

My thesis is to describe how well different subsequences of a gene sequence can identify a bacterial species in the human bladder. For the most part, the gene is the 16S ribosomal gene _rrn_, but there are other genes that can be used. I'm using the _rrn_ gene in this example. Any subsequences can be used, but the only practical ones are the result of a polymerase chain reaction (PCR) using different primers that anneal to the template. When you look at the _rrn_ sequence of a collection of bacterial species, the subsequences are not in the same place due to the different evolutionary history of the species. Insertions and deletions move the location of the different species' _rrn_ subsequences relative to each other. 

A really important step is to locate where the same subsequences are across those species of interest, and extract them. First, I'll create a multisequence alignment (MSA) of the _rrn_ gene, which ensures that the parts of the _rrn_ gene that are the same across the set of species are in the same location. 

![](pictures/msa.png)

This is a screen capture of the program UGENE, which I use because it was the easiest to figure out, but mostly because it's free. In the thin strip between the alignment window and the species names is the position on each respective _rrn_ gene. It's clear (if you squint at the little numbers at the top) that while the position of the MSA is 1286 base pairs from the beginning, the position in each species' _rrn_ sequence is different. Some positions are almost 100bp different from others.

![](pictures/mag.png)

In the next step, I map the location of the primers to the MSA.

![](pictures/amps.png)

Finally, I'll extract the subsequence bracketed by each primer from each respecive species' _rrn_ sequence. 

While I've been showing pictures to illustrate these examples, in reality they're just text files formatted in a particular way. All these steps have been operations on strings of characters. The trouble begins when I try to find the position of the base pair in the MSA to the corresponding position in the individual _rrn_ sequences of each species. I did this by hand, at first, by using the _rrn_ seqeunce of _E. coli_ as a reference for the primer locations, and transposing to the MSA coordinate by scrolling across the display, and then feeding that into a program function from the BioPython module. It's better when a computer does all the steps.

Extracting the sequences from a MSA is easy. A BioPyton module `AlignIO` will subset a MSA just like you would use slice notation on a string:

That's all there is to it. `msa` is the multisequence alignment file object created by `AlignIO`. The rows and columns of the MSA are seperated by a comma. The start and stop are indicated by numbers (or variables set to numbers). So the code above means use all the rows in the `msa`, but extract the column positions from `start_pos` to `end_pos`. The entire python script could just be this one line, but that's not good enough. The remainder of this presentation is about how I explain how to describe what the script does (documentation), what resources it needs (command line arguments), and how I make sure the script does what it's supposed to (unit tests), even when someone else uses it and tries to pass all kinds of garbage to it. Because fixing bugs through code inspection is like crawing through fiberglass insulation in a swimsuit, even when you wrote the code yourself. Even a small set of unit tests and some documentation makes it easier to maintain your code, and easier for someone else to use it.

## Unit tests

I found writing tests is a habit that must be reinforced. Test driven development is not as much fun as getting the code to work, and the number of test posibilities are open ended. I have many times when I cheat, and just bash out some code because I make excuses about how hard it is to think of relevant tests. However, once it's done I get code that has I know will work for the easy exceptions that I can think of, and any new bugs can be added to the test suite. For this script, I was all ready to write the unit tests first. Up to now, I've only done tests on functions, but this script didn't have any. I had to learn how to use the `subprocess` module first, which is a way to call a process (like the shell command `ls`, or even NCBI's `blastn`) from inside a python script. So I worked on validating the input files first. 

However, I can show how to do a small test suite on a silly function. 

### modules used

I'm going to import all the modules that I use in this notebook here at the beginning.

* [unittest](https://docs.python.org/3/library/unittest.html) - supports test automation, sharing of setup and shutdown code for tests, aggregation of tests into collections, and independence of the tests from the reporting framework.
* [argparse](https://docs.python.org/3/library/argparse.html?highlight=argparse#module-argparse) - this module makes it easy to write user-friendly command-line interfaces.
* [Biopython](https://biopython.org/) - a distributed collaborative effort to develop Python libraries and applications which address the needs of current and future work in bioinformatics.
* [io](https://docs.python.org/3/library/io.html?highlight=io#module-io) - This module provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter. It is always available.
* [sys](https://docs.python.org/3/library/sys.html?highlight=sys#module-sys) - 
* [os](https://docs.python.org/3/library/os.html?highlight=os#module-os) - This module provides a portable way of using operating system dependent functionality.
* [datetime](https://docs.python.org/3/library/datetime.html?highlight=datetime#module-datetime) - this module supplies classes for manipulating dates and times.
* [shutil](https://docs.python.org/3/library/shutil.html?highlight=shutil#module-shutil) - This module offers a number of high-level operations on files and collections of files. In particular, functions are provided which support file copying and removal.
* [subprocess](https://docs.python.org/3/library/subprocess.html?highlight=subprocess#module-subprocess) - This module allows you to spawn new processes, connect to their input/output/error pipes, and obtain their return codes. 

In [1]:
import unittest
import argparse
from Bio import SeqIO
from Bio import AlignIO
import io
import sys
import os
from datetime import datetime
import shutil
import subprocess

# this is a script that I wrote
import extra_functions

### testing Hello World

Tests are grouped into a single class. When the test script is run, `unittest` will look for anything starting with the word 'test' and run it automatically. So far I've written tests for each collection of functions that I use in my workflow. An easy test I like to include first is to check that the file is where I think it is. It's a stupid test, but I move files around sometimes. A failed test might be because it can't find the file in the first place, not because the tested code is at fault. The next test is to check that the function has actually printed "Hello World!" to the standard output, which is the terminal window. It's a good example of how the desire to do something simple turns into an hour of discovering how many layers are between the code I write and where the output ends up. But now I know, so in the future I can apply it to some other test.

The bulk of the tests I've written are based on assertions. Assert that these two values are equal. Assert that this boolean is 'True'. Assert that the file exists. That kind of thing. If you do nothing else with unit tests, assertions will still take you far.

With test driven development, you're supposed to start with a piece of paper or whiteboard. Then you write down what the code is supposed to accomplish, and with what resources (arguments, files, data, etc.). Then you conceive of a bunch of tests that will verify that the code will do that. Then you write the code. Once the code passes all the test you have written, you _stop writing code_. Back away from the keyboard, you're all done. 

I have yet to do this. I do start with paper, and fill it with boxes and arrows, and then write some tests. Then I'll write the code, and run the test. By then, I'll have realized that I need to add some other extra feature. So I'll put that in, and write a test for it. Rinse and repeat. The improvement over what I used to do is that the code still does what I had originally intended, even after extensive refactoring (rewriting code that's more concise is called 'refactoring'). 

In [2]:
class test_simple(unittest.TestCase):
    """
        A test suite
        
        All test functions must start with 'test' in order 
        for unittest to automatically find and run 
    """
    def test_script_exists(self):
        """
            make sure you can find the file
        """
        get_files=os.listdir("./")
        self.assertIn("extra_functions.py", get_files)
        
    def test_output(self):
        """
            checking the printed output was ridiculously difficult to do. 
            Thanks alot, python.
        """
        # create a stringIO object
        catch=io.StringIO()
        # redirect the sys.stdout
        sys.stdout=catch
        # call the function
        extra_functions.hello_world()
        # put the stdout back where it's supposed to be
        sys.stdout=sys.__stdout__
        #print(catch.getvalue())
        # assert that the caught output is what it's supposed to be
        self.assertEqual(catch.getvalue().strip(), 'Hello World!')

# this is how I got unittest to run inside the notebook
unittest.main(argv=[''], verbosity=2, exit=False)

test_output (__main__.test_simple) ... ok
test_script_exists (__main__.test_simple) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


You might have noticed that I put a lot of comments in `test_output()`, but none in `test_script_exists()`. If I'm doing something new, like redirecting where the output is going, then I'll put as many comments as needed so that my future self can figure out what I was doing. If I think it's already clear, like in the two lines of `test_script_exists()`, I won't. I've also gotten in the habit of writing a decent docstring, which is everything between the triple quotes. All that text can be displayed with a call to the `__doc__` method. I can't get it to format correctly in this notebook, but if you hit `shift+tab` a window will popup and display the docstring.

In [3]:
extra_functions.hello_world.__doc__

"\n        Just like the name suggests, this\n        trusty function will print \n        'Hello World!' and then sit quietly\n        with its hands in its lap.\n    "

## Documentation and command line flags with `argparse`

I've found it's a good idea to write out as much of the documentation early, such as in a docstring, but many times I didn't see a point because it was hard to show all that documentation where it was needed - at the command line, where I was getting stuck. I would have to scroll through the code or worse, follow a trail to the script in some other directory. Then I stumbled across the `argparse` module. I really like `argparse`. Like a shell program such as `grep`, `argparse` lets you tell the script to use a particular file, with a particular context, and then send the output to whatever directory you want. I used to do this in a small way by using `sys.argv` from the `sys` module, but I think `argparse` is much better.

### Script and function documentation

The basic format that I've settled on is to write a detailed description of what the script does in the `description` argument, then add as many optional or positional arguments as needed. Setting the argument `formatter_class=argparse.RawDescriptionHelpFormatter` allows me to format the text as I want it to display, which is easier than fussing with `\t` or `\n` characters. Then I list all the functions that I wrote that the script needs in the `epilog` argument. I don't know how to set the formatting in this part, so I use the `format` method of the string object. See the `extra_functions.hello_world.__doc__`? You might have to scroll to the right. That's where calling the docstring in a function comes in handy. 

Showing the documentation is done by adding a `--help` or `-h` flag after calling the script:

In [4]:
parsing=argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, description="""
         A long and detailed description of what this script will do. 
         And an example of how to run it from the command line, 
         because you know you'll forget.
         
             >python dummy.py -i /path/to/foo.txt
""", epilog="I like to list the docstrings of functions that I wrote myself here,\nespecially ones that live somewhere else in the directory tree:\n\n  hello_world()\n{}".format(extra_functions.hello_world.__doc__))

parsing.add_argument("--infile", "-i", required=True, help="path to the infile")
parsing.add_argument("--verbose", "-v", action="store_true", help="spell out what is happening")

# this is how I got argparse to work in the notebook, without calling a script
# in a script, I would type:
#    args=parsing.parse_args()

args=parsing.parse_args(['-h'])

SystemExit: 0

C:\Users\Carter\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Options

You get a `--help` option for free, but it's up to you to write something helpful. Anything beyond that is added by typing another `parsing.add_argument(args)` line. What you call this new option is entirely up to you. `--mike_wazowski` is fine. So is `--blank`. There are many arguments that you can set in the `.add_argument` method, and I have yet to use them all. Some examples are default values, whether the option is required, and what data type the option is. The most important in this context is the `help` argument. This is where you describe what the function of that option is. 

I commonly use three options on a regular basis.

* `--infile` or `--infolder` The path to the file that the script will work on, or the folder where a batch of files will be used

* `--outfile` The path to where I want output written to

* `--verbose` I tend to use a lot of `print` statements when I develop code. Instead of going back and deleting them, or commenting them out, I put them in an `if` statement and check if the `--verbose` option is used. Like this:

The output of running the cell above doesn't show up in the notebook (and it seems to throw an error), but if you look at the terminal where Jupyter is running, you should see this:

![](pictures/terminal_output.png)

Here's an example of one script where I went all in with the `--help` documentation, including a lush ASCII diagram.

![](pictures/long_term.png)

## Validate data with `try...except` clauses 

I need the script to extract parts of a MSA pretty much exactly where I tell it to. Upstream work has generated files using the Clustal format, and I tend to belive that it's done right. But, I might have to write data in a Clustal format on my own in the future, and that means I'll probably get it wrong. I use `try...except` clauses to validate the format, but I have to admit that for now I'm depending on BioPython to do the validating.

The `try...except` clause is a method of gracefully handling errors that you suspect might happen. Instead of the python interpreter killing the whole script and printing an error message, the clause allows you to tell the interpreter what to do next. Maybe, it's nothing more than writing your own custom error output. Maybe it's only going to affect one variable, and you can set it to something else. In general, the rule is that if your script is receiving data from somewhere else, it should absolutely positively make sure that the data is what is expected.

I made a few Clustal files for the unit tests and in-script validation. First was a correctly formatted clustal file:

#### good_clustal_format.aln

And an incorrectly formatted file, where just the header is not to spec:

#### bad_clustal_format.aln

Here's what happens when BioPython comes across a malformed Clustal file.

In [ ]:
# I'm using the infile option (-i) and a path to the badly formatted Clustal file
args=parsing.parse_args(['-i', 'bad_clustal_format.aln'])
msa = AlignIO.read(args.infile, "clustal")

`AlignIO` will expect a vaild Clustal file, and will throw an error with a lot of text when it comes across anything else. The type is a `ValueError`, and I'll look for that error type in a `try...except` clause, and later in an assertion test when I run the test suite.

In the code below, nothing happens because BioPython agrees that the format of the file meets the Clustal specs. I've also introduced a `try...except` clause to handle the error thrown by `AlignIO`.

In [ ]:
args=parsing.parse_args(['-i', 'good_clustal_format.aln'])

try:
    msa = AlignIO.read(args.infile, "clustal")
except ValueError or AssertionError:
    print("\n\tThe format of the file \"{}\" doesn't meet the Clustal file specifications:\n\n\t1) The first line in the file must start with the words \"CLUSTAL W\" or \"CLUSTALW\". Other information\n\t   in the first line is ignored.\n\t2) One or more empty lines.\n\t3) One or more blocks of sequence data. Each block consists of:\n\t\t* One line for each sequence in the alignment. Each line consists of:\n\t\t\t* the sequence name\n\t\t\t* white space\n\t\t\t* up to 60 sequence symbols.\n\t\t\t* optional - white space followed by a cumulative count of residues for the sequences\n\t\t* A line showing the degree of conservation for the columns of the alignment in this block\n\t\t (this can be left as blank. \" \" indicates no match, but for this application\n\t\t the conservation isn't important)\n\t\t* One or more empty lines.\n\n\tQuitting now.".format(args.infile))
    sys.exit(1)

First, the python interpreter will try to execute the code in the `try` portion of the clause. If all goes well, the interpreter exits the clause and continues with the next block of code. If an error does occur in the `try` portion, the interpreter moves to the `except` portion and executes the code there. Most importantly, it will attempt to continue on to the next code block, regardless of what error has happened (unless that error includes exiting the script). This is why the clause is usefull. Errors that would normally stop the script can be circumvented. The downside is that the clause will suppress any error messages, so I need some other way to signal the test suite that some error has happened. `sys.exit()` is a good way to do this.

In the `try` portion, I attempt to have `AlignIO` read in a Clustal file. If things are fine, the interpreter moves on to the next code block. If not, I have the interpreter look at the type of error `AlignIO` has thrown. If it's a `ValueError` or an `AssertionError`, the `sys` module will force an exit from the script and send a return code of 1. The convention of what number to use for the status of a program when it terminates is listed below.

* Exit status 0 = successfully run, all is well
* Exit status 1 = failure, as defined by the program
* Exit status 2 = command line useage failure

I want this to happen. I'm using the clause to print an error message I wrote myself explaining that the file is no good, and I don't want the script to continue to the next code block. I could have defined a custom message for these errors, but I'm using the value of the exit code for the test suite.

Passing a malformed file will trigger the error message and exit the script. Plus you should be able to see that the `SystemExit` is equal to 1.

In [ ]:
args=parsing.parse_args(['-i', 'bad_clustal_format.aln'])

try:
    msa = AlignIO.read(args.infile, "clustal")
except ValueError or AssertionError:
    print("\n\tThe format of the file \"{}\" doesn't meet the Clustal file specifications:\n\n\t1) The first line in the file must start with the words \"CLUSTAL W\" or \"CLUSTALW\". Other information\n\t   in the first line is ignored.\n\t2) One or more empty lines.\n\t3) One or more blocks of sequence data. Each block consists of:\n\t\t* One line for each sequence in the alignment. Each line consists of:\n\t\t\t* the sequence name\n\t\t\t* white space\n\t\t\t* up to 60 sequence symbols.\n\t\t\t* optional - white space followed by a cumulative count of residues for the sequences\n\t\t* A line showing the degree of conservation for the columns of the alignment in this block\n\t\t (this can be left as blank. \" \" indicates no match, but for this application\n\t\t the conservation isn't important)\n\t\t* One or more empty lines.\n\n\tQuitting now.".format(args.infile))
    sys.exit(1)

If you look at the notebook terminal window, you should see this output:

![](pictures/exit1.png)

This is a win. I put several of these `try...except` clauses throughout my script, and I'll use the exit codes for the test suite.

## Unittests for `extract_demo.py`

Finally, this is the full test suite I have for the `extract_demo.py` script. There's one last test file to add, and that's the check for a reference sequence from _E. coli_.

#### no_ref_seq.aln

In [5]:
class test_ambitious(unittest.TestCase):
    """
        subprocess.run() is new to me. Call the function and any 
        options as a list of arguments. 

        I had a lot of problems at first but figured it out.
            
        The paths to the different files were confusing. 
        Settled on using the cwd="where/is/foo.bar" to change the working directory 
        to the right /src_files dir, then point to the testing file by relative path.
            
        BioPython got confused when an argument in the subprocess list was like this:
            
            subprocess.run(["python", "extract_demo.py", "-i bad_clustal_format.aln"]
                                                                  ^
            
        There's something going on in the Windows environment that makes the OS 
        look for a file named " foo.bar", a file with a space as the first character.
        Changing that to split up the flag from the path fixed that:
            
            subprocess.run(["python", "extract_demo.py", "-i", "bad_clustal_format.aln"]
                                                               ^
                    
        Now you know.
    """
    def test_script_exists(self):
        """
            check the script exists
        """
        get_files=os.listdir("./")
        self.assertIn("extract_demo.py", get_files)
        
    def test_find_resources(self):
                
        get_files=os.listdir("./")
        self.assertIn("good_clustal_format.aln", get_files)
        self.assertIn("bad_clustal_format.aln", get_files)
        self.assertIn("no_ref_seq.aln", get_files)
        
    def test_alignio_hates_infile(self):
        """
            Since you're checking for a return code from sys.exit(), 
            make sure it's not confused with an exit code of 1 coming from 
                1) some other complaint raised by subprocess 
                2) some screwup in the tested script
        """

        checkit=subprocess.run(["python", "extract_demo.py", "-i", "bad_clustal_format.aln"], capture_output=True)

        # in the future, if you need to search the error message 
        # for something like 'ValueError:' you can use regex and assertTrue()
        # but make sure that the stderr is a string by setting text=True
        #value_error=re.compile('ValueError:')
        #self.assertTrue(value_error.search(checkit.stderr))
        
        # try...except will suppress the error output.
        # good thing I used a return code
        self.assertEqual(checkit.returncode, 1)
        
    def test_no_ref_seq(self):
        """
            return code should be 1 if there's no E. coli reference sequence. 
            Either CP016007.2543965.2545520 or CP016007.3589827.3591382
        """
        
        checkit=subprocess.run(["python", "extract_demo.py", "-i", "no_ref_seq.aln"], capture_output=True)
        
        self.assertEqual(checkit.returncode, 1)
        
    def test_check_output(self):
        """
            the output goes to /processed_data, which is nice
            
            probably not the best way to test, but since the extracted sequences should only be Ts, 
            then asserting that the set("TTTTT") is equal to the set("T"). Both should be just T.
        """
        
        checkit=subprocess.run(["python", "extract_demo.py", "-i", "good_clustal_format.aln", "-e", "5,13"], capture_output=True, text=True)
        
        print("\n\targs={}\n\treturn code={}\n\tstdout={}\n\tstderr={}".format(checkit.args, checkit.returncode, checkit.stdout, checkit.stderr))
        
        spl_output=checkit.stdout.strip().split("/")
        new_file="processed_data/{0}/{1}".format(spl_output[-2], spl_output[-1])
        
        for x in SeqIO.parse(new_file, "fasta"):
            self.assertEqual(set(x), set("T"))
          
        # clean up by removing the directory and all files in it
        shutil.rmtree("processed_data/{}".format(spl_output[-2]))
        
        
# this is how I got unittest to run inside the notebook
unittest.main(argv=[''], verbosity=2, exit=False)

test_alignio_hates_infile (__main__.test_ambitious) ... ok
test_check_output (__main__.test_ambitious) ... ok
test_find_resources (__main__.test_ambitious) ... ok
test_no_ref_seq (__main__.test_ambitious) ... ok
test_script_exists (__main__.test_ambitious) ... ok
test_output (__main__.test_simple) ... ok
test_script_exists (__main__.test_simple) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.895s

OK
